# Import and Install requirements

In [9]:
!python -m pip install --user tf-keras 

  Using cached tf_keras-2.18.0-py3-none-any.whl (1.7 MB)
  Using cached tensorflow-2.18.0-cp310-cp310-win_amd64.whl (7.5 kB)


You should consider upgrading via the 'c:\Users\lenovo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
!python -m pip install datasets, pydantic, outlines

  Using cached requests-2.32.3-py3-none-any.whl (64 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
zenml 0.60.0 requires requests<2.32.0, but you have requests 2.32.3 which is incompatible.
mergoo 0.0.9 requires accelerate~=0.27.2, but you have accelerate 0.28.0 which is incompatible.
mergoo 0.0.9 requires tqdm==4.66.2, but you have tqdm 4.66.6 which is incompatible.
apache-airflow 2.9.1 requires alembic<2.0,>=1.13.1, but you have alembic 1.8.1 which is incompatible.
apache-airflow 2.9.1 requires sqlalchemy<2.0,>=1.4.36, but you have sqlalchemy 2.0.31 which is incompatible.
You should consider upgrading via the 'c:\Users\lenovo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
from outlines import models, generate
from pydantic import BaseModel
import json
import numpy as np 
import pandas as pd 

# Specify rule and data for finetuning and few-shot learning

In [60]:
# 8 is CN btw
rule = """
A day will start from 7AM to 5PM
Early morning is defined as 7-9 AM,
Morning is defined as 9-12 AM,
Afternoon is defined as 12-3 PM,
Evening is defined as 3-5 PM,
Do not include days where the customer has a meeting booked.
"""
data = [
    {
        "input": {"customerID": "A01", "response": "I am available every morning from 9.am to 11.am, except on Wednesdays."},
        "output": '{"customerID": "A01", "original": "I am available every morning from 9.am to 11.am, except on Wednesdays.", "available_time": {"2": [9, 10, 11], "3": [9, 10, 11], "4":[], "5": [9, 10, 11], "6": [9, 10, 11], "7": [9, 10, 11], "8": [9, 10, 11]}, "prefer": "None"}'
    },
    {
        "input": {"customerID": "A02", "response": "I am available every afternoon from 1.pm to 3.pm, except on Fridays."},
        "output": '{"customerID": "A02", "original": "I am available every afternoon from 1.pm to 3.pm, except on Fridays.", "available_time": {"2": [13, 14, 15], "3": [13, 14, 15], "4": [13, 14, 15], "5":[], "6": [13, 14, 15], "7": [13, 14, 15], "8": [13, 14, 15]}, "prefer": "None"}'
    },
    {
        "input": {"customerID": "A03", "response": "I am available every evening from 6.pm to 8.pm, except on Mondays."},
        "output": '{"customerID": "A03", "original": "I am available every evening from 6.pm to 8.pm, except on Mondays.", "available_time": {"2":[], "3": [], "4": [], "5": [], "6": [], "7": [], "8": []}, "prefer": "None"}'
    },
    {
        "input": {"customerID": "B01", "response": "I am free on Tuesdays, but if possible, I prefer an early morning slot."},
        "output": '{"customerID": "B01", "original": "I am free on Tuesdays, but if possible, I prefer an early morning slot.", "available_time": {"3": [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]}, "prefer": {"3": [7, 8, 9]}}'
    },
    {
        "input": {"customerID": "B02", "response": "I already have a meeting booked on Friday from 2 to 4 PM."},
        "output": '{"customerID": "B02", "original": "I already have a meeting booked on Friday from 2 to 4 PM.", "available_time": {"2": [7, 8, 9, 10, 11, 12, 13, 17], "3": [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], "4": [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], "5": [7, 8, 9, 10, 11, 12, 13, 17], "6": [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], "7": [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], "8": [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]}, "prefer": "None"}'
    }
]

df = pd.DataFrame(data)
df.to_csv("training_data.csv", index=False)

# Finetune model and few-shot learning (Failed, sorry guys)

In [61]:
# import os
# from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
# from transformers import LineByLineTextDataset

# # Initialize the GPT-2 model and tokenizer
# model_name = "gpt2"
# model = GPT2LMHeadModel.from_pretrained(model_name)
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token  # Set EOS token as the padding token

# # Load the dataset
# def load_dataset(file_path):
#     # Load the data as line-by-line text for the training format
#     return LineByLineTextDataset(tokenizer=tokenizer, file_path=file_path, block_size=128)

# # Convert your JSONL data to plain text format for fine-tuning
# def convert_jsonl_to_text(jsonl_path, text_output_path):
#     with open(jsonl_path, "r") as jsonl_file, open(text_output_path, "w") as text_file:
#         for line in jsonl_file:
#             item = eval(line)
#             prompt = item["input"]
#             output = item["output"]
#             # Concatenate prompt and output for training
#             combined = f"{prompt}\n{output}\n\n"
#             text_file.write(combined)

# # Paths
# jsonl_path = "training_data.jsonl"
# text_output_path = "training_data.txt"
# convert_jsonl_to_text(jsonl_path, text_output_path)

# # Load the dataset
# train_dataset = load_dataset(text_output_path)

# # Set up the data collator
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="./gpt2-finetuned",
#     overwrite_output_dir=True,
#     num_train_epochs=3,
#     per_device_train_batch_size=2,
#     save_steps=500,
#     save_total_limit=2,
#     prediction_loss_only=True,
# )

# # Initialize the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=train_dataset,
# )

# # Train the model
# trainer.train()

# # Save the model and tokenizer
# model.save_pretrained("./gpt2-finetuned")
# tokenizer.save_pretrained("./gpt2-finetuned")


In [62]:
# from langchain_core.output_parsers import JsonOutputParser
# from langchain_core.prompts import PromptTemplate
# from langchain.llms import HuggingFacePipeline
# from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
# import json

# # Load the fine-tuned model and tokenizer
# model = AutoModelForCausalLM.from_pretrained("./gpt2-finetuned")
# tokenizer = AutoTokenizer.from_pretrained("./gpt2-finetuned")
# hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=500)

# llm = HuggingFacePipeline(pipeline=hf_pipeline)

# # Define JSON schema for LangChain’s JsonOutputParser
# json_schema = {
#     "customerID": "string",
#     "original": "string",
#     "available_time": {
#         "type": "object",
#         "properties": {
#             "2": {"type": "array", "items": {"type": "integer"}},
#             "3": {"type": "array", "items": {"type": "integer"}},
#             "4": {"type": "array", "items": {"type": "integer"}},
#             "5": {"type": "array", "items": {"type": "integer"}},
#             "6": {"type": "array", "items": {"type": "integer"}},
#             "7": {"type": "array", "items": {"type": "integer"}},
#             "CN": {"type": "array", "items": {"type": "integer"}}
#         }
#     },
#     "prefer": "string"
# }

# output_parser = JsonOutputParser(schema=json_schema)

# # Define a clearer prompt template for JSON generation
# rule = """
# A day will start from 7AM to 5PM.
# Early morning is defined as 7-9 AM,
# Morning is defined as 9-12 AM,
# Afternoon is defined as 12-3 PM,
# Evening is defined as 3-5 PM.
# Do not include days where the customer has a meeting booked.
# """

# prompt_template = PromptTemplate(
#     input_variables=["customerID", "response"],
#     template=(
#         f"Rule: {rule.strip()}\n\n"
#         "Customer ID: {customerID}\n"
#         "Customer Response: \"{response}\"\n\n"
#         "Please generate output JSON in the following structure exactly:\n"
#         "{{\n"
#         "    \"customerID\": \"{customerID}\",\n"
#         "    \"original\": \"{response}\",\n"
#         "    \"available_time\": {{\"2\": [], \"3\": [], \"4\": [], \"5\": [], \"6\": [], \"7\": [], \"CN\": []}},\n"
#         "    \"prefer\": \"None\"\n"
#         "}}"
#     )
# )

# # Function to generate JSON output with error handling
# def generate_json_output(customerID, response):
#     prompt = prompt_template.format(customerID=customerID, response=response)
#     result = llm(prompt)
    
#     # Extract the text generated by the model
#     generated_text = result.strip()

#     parsed_output = output_parser.parse(generated_text)


#     return parsed_output

# # Test with a sample input
# customer_id = "A01"
# response = "I am available every morning from 9.am to 11.am, except on Wednesdays."
# output = generate_json_output(customer_id, response)
# print(output)


# Scheduling algorithm

In this step, i assume that the LLM has already extract the json-string format correctly and beautifully (however it's not, i am fixing it, but i affraid that i can not do it on time). 

Now for each user, we will have a json file like this: 
{
    "customerID": "A01",
    "original": "I am available every morning from 9.am to 11.am, except on Wednesdays.",
    "available_time": {
        "2": [9, 10, 11],
        "3": [9, 10, 11],
        "4": [],
        "5": [9, 10, 11],
        "6": [9, 10, 11],
        "7": [9, 10, 11],
        "8": [9, 10, 11]
    },
    "prefer": "None"
}


I will use the output of the data above to use in this example because the model i trained fail badly :(

In [63]:
# Since the ouput of the model is a string so that we need to convert it to dictionary
all_customer = []
def convert_str_to_dict(string):
    return json.loads(string)
for output in df["output"]:
    customer_dict = convert_str_to_dict(output)
    all_customer.append(customer_dict)

In [64]:
all_customer

[{'customerID': 'A01',
  'original': 'I am available every morning from 9.am to 11.am, except on Wednesdays.',
  'available_time': {'2': [9, 10, 11],
   '3': [9, 10, 11],
   '4': [],
   '5': [9, 10, 11],
   '6': [9, 10, 11],
   '7': [9, 10, 11],
   '8': [9, 10, 11]},
  'prefer': 'None'},
 {'customerID': 'A02',
  'original': 'I am available every afternoon from 1.pm to 3.pm, except on Fridays.',
  'available_time': {'2': [13, 14, 15],
   '3': [13, 14, 15],
   '4': [13, 14, 15],
   '5': [],
   '6': [13, 14, 15],
   '7': [13, 14, 15],
   '8': [13, 14, 15]},
  'prefer': 'None'},
 {'customerID': 'A03',
  'original': 'I am available every evening from 6.pm to 8.pm, except on Mondays.',
  'available_time': {'2': [],
   '3': [],
   '4': [],
   '5': [],
   '6': [],
   '7': [],
   '8': []},
  'prefer': 'None'},
 {'customerID': 'B01',
  'original': 'I am free on Tuesdays, but if possible, I prefer an early morning slot.',
  'available_time': {'3': [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]},
  'pr

In [80]:
# This function will create a dataframe that represents the availability of each customer, it will add 0.01 to each customer time if they prefer it
def create_availability_dataframe(customers_data):
    # Dictionary to store each customer's availability DataFrame, with customerID as the key
    availability_dict = {}

    for customer in customers_data:
        customer_id = customer['customerID']
        available_time = customer['available_time']
        prefer = customer['prefer']
        
        # Initialize individual DataFrame for each customer
        df_availability = pd.DataFrame(0, index=range(2, 9), columns=range(7, 18))

        for day, hours in available_time.items():
            for hour in hours:
                # Set availability
                df_availability.at[int(day), int(hour)] = 1
                
                # Check and apply preference if it's a dictionary
                if isinstance(prefer, dict) and day in prefer:
                    if int(hour) in prefer[day]:
                        df_availability.at[int(day), int(hour)] += 0.01
        
        # Store the DataFrame in the dictionary with customerID as the key
        availability_dict[customer_id] = df_availability

    return availability_dict
demo = create_availability_dataframe([all_customer[0]])
demo


{'A01':    7   8   9   10  11  12  13  14  15  16  17
 2   0   0   1   1   1   0   0   0   0   0   0
 3   0   0   1   1   1   0   0   0   0   0   0
 4   0   0   0   0   0   0   0   0   0   0   0
 5   0   0   1   1   1   0   0   0   0   0   0
 6   0   0   1   1   1   0   0   0   0   0   0
 7   0   0   1   1   1   0   0   0   0   0   0
 8   0   0   1   1   1   0   0   0   0   0   0}

In [81]:
all_customer_schedule = [create_availability_dataframe([customer]) for customer in all_customer]

C:\Users\lenovo\AppData\Local\Temp\ipykernel_12984\69011362.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_availability.at[int(day), int(hour)] += 0.01
C:\Users\lenovo\AppData\Local\Temp\ipykernel_12984\69011362.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_availability.at[int(day), int(hour)] += 0.01
C:\Users\lenovo\AppData\Local\Temp\ipykernel_12984\69011362.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_availability.at[int(day), in

In [85]:
all_customer_schedule

[{'A01':    7   8   9   10  11  12  13  14  15  16  17
  2   0   0   1   1   1   0   0   0   0   0   0
  3   0   0   1   1   1   0   0   0   0   0   0
  4   0   0   0   0   0   0   0   0   0   0   0
  5   0   0   1   1   1   0   0   0   0   0   0
  6   0   0   1   1   1   0   0   0   0   0   0
  7   0   0   1   1   1   0   0   0   0   0   0
  8   0   0   1   1   1   0   0   0   0   0   0},
 {'A02':    7   8   9   10  11  12  13  14  15  16  17
  2   0   0   0   0   0   0   1   1   1   0   0
  3   0   0   0   0   0   0   1   1   1   0   0
  4   0   0   0   0   0   0   1   1   1   0   0
  5   0   0   0   0   0   0   0   0   0   0   0
  6   0   0   0   0   0   0   1   1   1   0   0
  7   0   0   0   0   0   0   1   1   1   0   0
  8   0   0   0   0   0   0   1   1   1   0   0},
 {'A03':    7   8   9   10  11  12  13  14  15  16  17
  2   0   0   0   0   0   0   0   0   0   0   0
  3   0   0   0   0   0   0   0   0   0   0   0
  4   0   0   0   0   0   0   0   0   0   0   0
  5   0   0   0

The meaning is that after we sum all the customer schedule, we will also get a dataframe with 7x10 (7 days x 10 hours). Each value in this dataframe will be a number ranging from 0 (no customer available at that time) to n_customer*(1.01) (every customer available at that time and like that time). 

In [ ]:
# We can create a function to select the time slots where at least a certain number of customers are available
def find_available_slots(all_customer_schedule, min_customers):
    combined_availability = pd.DataFrame(0, index=range(2, 9), columns=range(7, 18))

    availability_tracker = {}

    for customer_dict in all_customer_schedule:
        for customer_id, df in customer_dict.items():
            # Add availability data to the combined availability matrix
            combined_availability += df
            for day, row in df.iterrows():
                for time, availability in row.items():
                    if availability > 0:  # customer is available at this time slot
                        if (day, time) not in availability_tracker:
                            availability_tracker[(day, time)] = []
                        availability_tracker[(day, time)].append(customer_id)

    available_slots = combined_availability[combined_availability >= min_customers].stack()

    results = []
    for slot in available_slots.index:
        day, time = slot
        customers = availability_tracker.get((day, time), [])
        results.append((day, time, customers))
        print(f"Day {day}, Time {time}: Available Customers - {customers}")

    return results

# Example usage
min_customers = 3
available_slots = find_available_slots(all_customer_schedule, min_customers)



Day 3, Time 9: Available Customers - ['A01', 'B01', 'B02']
Day 3, Time 10: Available Customers - ['A01', 'B01', 'B02']
Day 3, Time 11: Available Customers - ['A01', 'B01', 'B02']
Day 3, Time 13: Available Customers - ['A02', 'B01', 'B02']
Day 3, Time 14: Available Customers - ['A02', 'B01', 'B02']
Day 3, Time 15: Available Customers - ['A02', 'B01', 'B02']


Look at the results, we can see that on tuesdays, at least 3 members can join the meeting from 9am to 11am and 1pm to 3pm.

In [89]:
min_customers = 2
available_slots = find_available_slots(all_customer_schedule, min_customers)

Day 2, Time 9: Available Customers - ['A01', 'B02']
Day 2, Time 10: Available Customers - ['A01', 'B02']
Day 2, Time 11: Available Customers - ['A01', 'B02']
Day 2, Time 13: Available Customers - ['A02', 'B02']
Day 3, Time 7: Available Customers - ['B01', 'B02']
Day 3, Time 8: Available Customers - ['B01', 'B02']
Day 3, Time 9: Available Customers - ['A01', 'B01', 'B02']
Day 3, Time 10: Available Customers - ['A01', 'B01', 'B02']
Day 3, Time 11: Available Customers - ['A01', 'B01', 'B02']
Day 3, Time 12: Available Customers - ['B01', 'B02']
Day 3, Time 13: Available Customers - ['A02', 'B01', 'B02']
Day 3, Time 14: Available Customers - ['A02', 'B01', 'B02']
Day 3, Time 15: Available Customers - ['A02', 'B01', 'B02']
Day 3, Time 16: Available Customers - ['B01', 'B02']
Day 3, Time 17: Available Customers - ['B01', 'B02']
Day 4, Time 13: Available Customers - ['A02', 'B02']
Day 4, Time 14: Available Customers - ['A02', 'B02']
Day 4, Time 15: Available Customers - ['A02', 'B02']
Day 5, T

Sending invitation or verify user is for further investigation. This notebook shows that we can create a scheduling system just like that.